## How LambdaMART works

Here is the high-level workflow of xgboost LambdaMart:

### 1. **Initial Model and Baseline Prediction**
- The process begins with an initial model that makes baseline predictions about the relevance of items. This model can be very simple, often just predicting the average relevance score.

### 2. **Calculation of Gradients (Lambdas)**
- After the initial predictions, the algorithm calculates gradients, which, in ranking tasks, are represented by lambda values. These lambdas are derived from the difference in a ranking metric (like NDCG) that would result from swapping pairs of items in the ranking. They quantify the direction and magnitude of the error in the ranking.

### 3. **Building Weak Learners to Predict Gradients**
- Gradient boosting then builds weak learners (typically decision trees) to predict these gradients rather than directly predicting the relevance scores. Each tree focuses on correcting the errors made by the ensemble of previously built trees.

### 4. **Adjusting Predictions**
- The predictions from the model are adjusted based on the output of these weak learners, effectively moving the predicted ranking closer to the optimal ranking. The adjustments are made in a direction that improves the overall ranking metric, with the magnitude of adjustment modulated by the learning rate.

### 5. **Iterative Improvement**
- This process of calculating gradients, building trees to predict these gradients, and adjusting the model's predictions is repeated iteratively. With each iteration, the model becomes better at predicting the correct ranking order of items.

### 6. **Ensemble of Weak Learners**
- The final model is an ensemble of all the weak learners built throughout the iterations. This ensemble model is capable of accurately predicting the relevance of items, resulting in a ranking that maximizes the chosen ranking metric.



## How lambdas are calculated

To provide a more concrete example with step-by-step numerical calculations for the LambdaMART algorithm using NDCG as the metric, let's visit a simplified scenario with 3 items: A, B, and C. We'll include the actual formulas for NDCG and show how lambda values are calculated and used to adjust the model's scores.

### Simplified Scenario Setup
- **Items and their true relevance scores**: A=3, B=2, C=1
- **Initial Model Scores** leading to the ranking: C > B > A (an incorrect ranking based on scores, not relevance)

### Objective
Optimize the ranking to match true relevance: **Ideal Ranking**: A > B > C

### Step 1: Calculate Initial NDCG
NDCG is calculated using the formula:
$$ NDCG@k = \frac{DCG@k}{IDCG@k} $$

where
$$ DCG@k = \sum_{i=1}^{k} \frac{2^{rel_i} - 1}{\log_2(i + 1)} $$
$ IDCG@k $ is the DCG score of the ideal ranking.

For simplicity, let's consider NDCG@3 for all items.

#### Initial DCG
Given the initial ranking (C > B > A), let's calculate DCG@3:
$$ DCG@3 = \frac{2^1 - 1}{\log_2(2)} + \frac{2^2 - 1}{\log_2(3)} + \frac{2^3 - 1}{\log_2(4)} $$

#### Ideal DCG (IDCG)
For the ideal ranking (A > B > C):
$$ IDCG@3 = \frac{2^3 - 1}{\log_2(2)} + \frac{2^2 - 1}{\log_2(3)} + \frac{2^1 - 1}{\log_2(4)} $$

Let's calculate these values.

With the calculations done:

- **Initial DCG** (for the incorrect ranking C > B > A): 6.39
- **IDCG** (for the ideal ranking A > B > C): 9.39
- **Initial NDCG**: 0.68

This NDCG value indicates the effectiveness of the initial ranking compared to the ideal ranking.

In [26]:
import numpy as np

def dcg(relevances):
    """Calculate Discounted Cumulative Gain (DCG)."""
    relevances = np.array(relevances)
    # We use np.arange to create a denominator that starts from 2 to len(relevances) + 1
    # because the log base 2 of 1 is 0, and we are trying to avoid DivisionByZero error.
    denominators = np.log2(np.arange(2, len(relevances) + 2))
    return np.sum((2**relevances - 1) / denominators)

def ndcg(predictions, relevances):
    """Calculate Normalized Discounted Cumulative Gain (NDCG)."""
    # Convert predictions to numpy array and sort them according to the predicted scores
    # Assuming higher scores should rank higher, we sort both predictions and relevances
    # based on predictions to simulate the ranking process.
    predictions = np.array(predictions)
    relevances = np.array(relevances)
    predicted_order = np.argsort(predictions)[::-1]
    sorted_relevances = relevances[predicted_order]
    
    # Calculate DCG for the predicted ranking
    predicted_dcg = dcg(sorted_relevances)
    
    # Calculate DCG for the ideal ranking
    ideal_relevances = np.sort(relevances)[::-1]
    ideal_dcg = dcg(ideal_relevances)
    
    # Avoid division by zero
    if ideal_dcg == 0:
        return 0
    
    # Calculate NDCG
    return predicted_dcg / ideal_dcg

# Example usage:
relevances = [3, 2, 1]  # True relevance scores
predictions = [1, 2, 3]  # Predicted relevance scores (model output)

ndcg_score = ndcg(predictions, relevances)
print(f"NDCG Score: {ndcg_score}")

NDCG Score: 0.6806060567602009


### Step 2: Calculate Lambda Values
Lambda values would be calculated based on the impact of swapping pairs on the NDCG. However, for simplicity, let's illustrate how lambda values might be adjusted conceptually, given the substantial difference between the initial and ideal NDCG values.

In [24]:
def calculate_loss_changes_with_labels(items, preds, values):
    """
    Calculate and document the changes in the loss function for all pairwise swaps, using item labels.

    Parameters:
    - items: List of item labels (e.g., ["A", "B", "C"]).
    - values: List of corresponding values for the items.

    Returns:
    - A list of dictionaries, each containing details of the swap (using labels) and the change in loss.
    """
    # Initial loss calculation
    initial_loss = ndcg(preds, values)
    
    def compute_loss(preds, values):
        """Helper function to compute loss for any given order of values."""
        return ndcg(preds, values)

    swaps_and_changes = []

    for i in range(len(items)):
        for j in range(i + 1, len(items)):
            # Swap the values corresponding to the items
            new_preds = preds.copy()
            new_preds[i], new_preds[j] = new_preds[j], new_preds[i]
            
            new_loss = compute_loss(new_preds, values)
            change = new_loss - initial_loss
            
            # Use item labels for documenting the swap
            swap_details = {
                'swap': f"{items[i]} <-> {items[j]}",
                'change': change
            }
            swaps_and_changes.append(swap_details)

    return swaps_and_changes

# Items and their values
items_example = ["A", "B", "C"]
preds_example = [1, 2, 3]
values_example = [3, 2, 1]

# Calculate and document swaps
result_with_labels = calculate_loss_changes_with_labels(items_example, preds_example, values_example)

# Example output
for r in result_with_labels:
    print(f"Swap: {r['swap']}, Change in Loss Function: {r['change']}")


Swap: A <-> B, Change in Loss Function: 0.05575756037413726
Swap: A <-> C, Change in Loss Function: 0.3193939432397991
Swap: B <-> C, Change in Loss Function: 0.07858586755953101


**Simplified Aggregation Approach:**

- Lambda for A: Sum of changes from swaps involving A (A ↔ B, A ↔ C).
- Lambda for B: Sum of changes from swaps involving B (A ↔ B, B ↔ C) but considering the direction of the effect.
- Lambda for C: Sum of changes from swaps involving C (A ↔ C, B ↔ C), again considering the direction.

Let's proceed with this simplified aggregation:

The aggregate lambda values, based on the simplified aggregation of NDCG changes due to swaps, are as follows:

- Lambda for A: +0.375
- Lambda for B: +0.023
- Lambda for C: -0.39


**Interpretation of Aggregate Lambda Values**

- A: The positive lambda value indicates a strong need to increase A's model score to move it towards its correct higher position in the ranking, reflecting its high relevance.

- B: The smaller positive lambda value for B suggests a moderate adjustment is needed, fine-tuning its position likely closer to its current one but ensuring it's correctly ordered relative to A and C.

- C: The negative lambda value for C indicates a need to decrease C's model score, correcting its initial over-ranking by moving it towards a lower position in the ranking.